# O4 genkendelse af manipulerede billeder, af ansigter

## Overvejser

### Datasættet
- Alle ansigter er centreret
- Mulighed for at fjerne detaljer i hjørner. Vi er meget specifikt intresseret i mund, næse & øjne
- G fold cross validation til at lave trænings og validerings sæt
- Alle billeder samme størrelse, derfor ikke brug for padding filter
- feature scaling (smid filter til at nedvurderer hjørner af billeder, da der sjælendt er ansigt i hjørnerne) #preproccesing

## Code and implementation

### Imports and frameworks
Here we import the necesarry libraries and frameworks which we will use in the generation of our machine learning model.

In [1]:
# Imports and other modules we use
import os # Directory and file reading
import re # Regex
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from IPython.display import Image as IImage
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# We should probably use keras since we want the make an CNN and possibly use our GPU
import keras
from keras import layers

### Constants and custom hyperparameters
Following are some of our custom hyperparameters which change some of the preprocessing changes we make to the data.

In [2]:
apply_filter = True # Should we apply a darkening border filter to the images. Since the faces are mostly centered.
test_size = 0.25 # Amount of test data. Value is normalized eg. 0.25 = 25%
#difficulties = ['easy'] # Could be used to choose what images we would like to

# Images are fetched from https://www.kaggle.com/ciplab/real-and-fake-face-detection

# !!!!!!!!!!!! make the paths relative from the notebook !!!!!!!!!!!!!!!
fake_dir = "C:/Projects/SWMAL/real_and_fake_face/training_fake" #Directory for fake images
real_dir = "C:/Projects/SWMAL/real_and_fake_face/training_real" #Directory for real images

### Generation of input array
This step fetches all images and loads them into the X array. All images are of the size 600x600 with RGB values from 0 to 255. This makes the images have the shape (600, 600, 3)


This array includes both the fake and the real images.

In [3]:
fake = os.listdir(fake_dir)
real = os.listdir(real_dir)
pattern = "(\w+)_(\d+)_(\d)(\d)(\d)(\d).jpg"


# Open images and insert into list
X = []  # We insert all images into the same list, as we use train_test_split and K-fold to split the data

# !!! This takes time !!!

for n in fake:
    result = re.match(pattern, n)
    if result.group(1) == 'easy':
        X.append(np.array(Image.open(fake_dir + "/"+ n)))

for n in real:#[0: 10]:
    X.append(np.array(Image.open(real_dir + "/"+ n)))

# Show example image
IImage(filename= fake_dir + "/" + fake[1])

# Be sure that the images are in a correct format (The first param (2041) might be removed or changed depending on the specific dataset)
print(np.shape(X))
#print(X)
#assert np.shape(X) == (2041, 600, 600, 3)
x_np = np.array(X)

(1321, 600, 600, 3)


### Generation of result array
Just as with the input array we generate an output array with the correct answers.
This includes which part of the image is edited. By default all real images will have [0,0,0,0] as the result otherwise the results adhear to the following syntax:

['left eye','right eye','nose','mouth']

All answers are in binary form. eg. [0,1,0,1] = Right eye and mouth changed

In [4]:
# Regex pattern for matching difficulty aswell as changed parts of the face
pattern = "(\w+)_(\d+)_(\d)(\d)(\d)(\d).jpg"

# Result array
y = []

# Array of difficulties
type_list = []

# Create results for fake images
dir_list = os.listdir(fake_dir)
for i in dir_list:
    result = re.match(pattern, i)
    if result.group(1) == 'easy':
        #acc = []
        #
        result = re.match(pattern, i)
        #left, right, nose, mouth = result.group(3, 4, 5, 6)
        #
        #acc.append(int(left))
        #acc.append(int(right))
        #acc.append(int(nose))
        #acc.append(int(mouth))
        type_list.append(result.group(1))
        #y.append(acc)
        y.append(1)

    
# Create result for real images
dir_list = os.listdir(real_dir)
for i in dir_list:
    #acc = []    
    #acc.append(0)
    #acc.append(0)
    #acc.append(0)
    #acc.append(0)
    #type_list.append("real")
    #y.append(acc)
    y.append(0)
    
y_np = np.array(y)

### Preprocessing of all data

Since most of the faces are already centered we would like the turn down some of the features of the corners, which we already know doesnt include very much interesting data about if the photo is manipulated or not.

Therefore we apply a filter which darkens the edges of the images. This is done to all images (both real and fake).

In [5]:
if apply_filter:
    # Create filter

    # Apply filter to all images

    # Results stay the same
    a = True # remove when actually code written

### Splitting the data into test and training data
First we want to randomly split the data into a training and a test set. This is done with train_test_split, where we use 25% of the data as test data.

After this we use K-fold with cross validation to try and get the most out of the remaining data.

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_np, y_np, test_size=test_size, random_state=42)

assert len(x_train) == len(y_train) and len(x_test) == len(y_test) 

print(f"""
training size:\t{len(x_train)}
test size:\t{len(x_test)}""")
print(y_test)

#generate the fold object
#kf = KFold(n_splits=2)
# The kfold object will return the indices of the splits in the training set.



training size:	990
test size:	331
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]


In [7]:
# Base model, LeNET5
seq = keras.Sequential()

seq.add(layers.Conv2D(filters=6, kernel_size=(3,3), activation='relu', input_shape=(600,600,3)))
seq.add(layers.AveragePooling2D())

seq.add(layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
seq.add(layers.MaxPooling2D())

seq.add(layers.Flatten())

seq.add(layers.Dense(units=60, activation='relu'))
seq.add(layers.Dense(units=30, activation='relu'))
seq.add(layers.Dense(units=1, activation='sigmoid'))

seq.summary()

seq.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#seq.compile(optimizer='sgd', loss=keras.losses.BinaryCrossentropy(), metrics=[keras.metrics.BinaryAccuracy()]) # optimizer='SGD', loss='MSE'

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 598, 598, 6)       168       
                                                                 
 average_pooling2d (AverageP  (None, 299, 299, 6)      0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 297, 297, 32)      1760      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 148, 148, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 700928)            0         
                                                                 
 dense (Dense)               (None, 60)                4

In [26]:
import tensorflow as tf
from tensorflow.keras import models, layers


data_aug = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
    
])


resize_and_rescale = keras.Sequential([
    layers.experimental.preprocessing.Resizing(600,600),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])
input_shape=(600,600,3)

model = models.Sequential([
    resize_and_rescale,
    data_aug,
    layers.Conv2D(32, (3,3), activation='relu', input_shape = input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    
    layers.Flatten(),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(3, activation= 'softmax'),
    
])

model.build(input_shape=input_shape)

mode.summary()

model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

ValueError: Input 0 of layer "conv2d_26" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (600, 600, 3)

In [ ]:
seq.fit(x_train, y_train, epochs=2)

In [ ]:
results = seq.predict(x_test)
print(results)

y_pred = []
for r in results:
    y_pred.append(r >=0.5)
#print(len(y_pred))
#print(len(y_test))

correct_guess = []
for index in range(0,len(y_pred)):
    print(f"""y_pred : {y_pred[index]}\ny_test:{y_test[index]}\ncompared:{y_pred[index] == y_test[index]}\n""")
    correct_guess.append(y_pred[index] == y_test[index])
    
#print(correct_guess)
print(seq.evaluate(x_test, y_test))

In [ ]:
y_pred.count(False)